In [8]:
import os
import re
import torch
import pandas as pd
import streamlit as st

import chromadb

from models import CustomEmbeddingFunction, Model
from db import fill_collection
from parser import Parser

# Queries and GTs

In [ ]:
queries = [
    'I am removing a gas tank. I have removed the main muffer. What is the next step?',
    'Rear seat belt warning lamp does not illuminate. How to diagnost it?',
    'I think my rear weels alignment is not right. What should I check?',
    'What are the steps to remove a coil spring?',
    'I am removing a steering weel. What should I do after removing air bag module?',
    'What is the first step in removing a steering column?',
    'How to disassemble steering gear and linkage?',
    'I removed a rear drive shaft. How should I inspect it after removal?',
    'I need to install am accelerator control system. What are the steps?',
    'There are squeaks in the centre console. What could be the reason of the noise?'
]

gts = [
    'Remove vent hose from filler tube',
    '''
Diagnosis Procedure
 1.CHECK DTC OF “AIR BAG DIAGNOSIS SENSOR UNIT”     
Check DTC in “Self Diagnosis Result” “Air bag diagnosis sensor unit” using CONSULT.
 Is DTC detected?
 YES >>Perform the trouble diagnosis related to the detected DTC. Refer to SRC-19, "
 DTC
 Index
 "
 .
 NO >>GO TO 2.
 2.CHECK DTC OF “COMBINATION METER” 
Check DTC in “Self Diagnosis Result” “Air bag diagnosis sensor unit” using CONSULT.
 Is DTC detected?
 YES >>Perform the trouble diagnosis related to the detected DTC. Refer to MWI-40, "
 DTC Index
 "
 .
 NO >>GO TO 3
 3.CHECK COMBINATION METER POWER SUPPLY AND GROUND CIRCUIT
 Check combination meter power supply and ground circuit. Refer to MWI-66, "
 COMBINATION METER : Diag
nosis Procedure
 "
 .
 Is the inspection result normal?
 YES >>GO TO 4.
 NO >>Repair or replace the malfunctioning parts.
 4.CHECK HARNESS CONNECTOR
 Check the harness connector for disconnection, looseness or damage.
 Is the inspection result normal?
 YES >>GO TO 5.
 NO-1 >>damage: Replace malfunctioning parts. 
NO-2 >>disconnection or looseness: Securely lock the connector.
 5.CHECK WIRING HARNESS
 Check the wiring harness externals.
 Is the inspection result normal?
 YES >>GO TO 6.
 NO >>Replace malfunctioning parts.
 6.CHECK REAR SEAT BELT BUCKLE SWITCH
 Check the applicable rear seat belt buckle switch. Refer to SBC-17, "
 Diagnosis Procedure
 "
 .
 Is the inspection result normal?
 YES >>GO TO 7.
 NO >>Replace the applicable rear seat belt buckle.
 7.REPLACE AIR BAG DIAGNOSIS SENSOR UNIT
 1. Replace air bag diagnosis sensor unit.
 2. Check air bag warning lamp operation.
 Is the inspection result normal?
 YES >>INSPECTION END
 NO >>GO TO 8.
 8.REPLACE COMBINATION METER
 1. Replace combination meter. Refer to MWI-86, "
 Removal and Installation
 "
 .
 2. Check air bag warning lamp operation.
 Is the inspection result normal?
 YES >>INSPECTION END
 NO >>GO TO 9.
 9.CHECK INTERMITTENT INCIDENT
 Refer to GI-44, "
 Intermittent Incident
 "
 .
 >> INSPECTION END
''',
'''
Preliminary Check
 Check the following:
 • Tires for improper air pressure and wear
 • Road wheels for runout: Refer to WT-60
 • Wheel bearing axial end play: Refer to RAX-3
 • Shock absorber operation
 • Each mounting point of axle and suspension for looseness and deformation
 (4WD).
 • Each of lower link, upper link, rear suspension member, suspension arm and shock absorber for cracks,
 deformation, and other damage
 • Vehicle height (posture)
 • Measure camber of both right and left wheels with a suitable alignment gauge.
 • If camber exceeds the standard value, adjust with adjusting bolt in
 lower. Refer to RSU-6, "
 • Measure toe-in by the following procedure.
''',
'''
1. Remove tires from vehicle.
 2. Remove torque member mounting bolts. Hang torque member where it does not interfere with work. Refer
 to BR-43, "
 BRAKE CALIPER ASSEMBLY : Exploded View
 "
 BLY : Exploded View
 "
 (RHD).
 (LHD), BR-86, "
 BRAKE CALIPER ASSEM
CAUTION:
 Never depress brake pedal while brake caliper is removed.
 3. Remove wheel sensor from axle housing. Refer to BRC-139, "
 REAR WHEEL SENSOR : Exploded View
 "
 .
 4. Set suitable jack under suspension arm.
 5. Remove shock absorber from suspension arm. Refer to RSU-9, "
 Exploded View
 "
 .
 6. Remove lower link from suspension arm. Refer to RSU-13, "
 Exploded View
 "
 .
 7. Remove upper link from suspension arm. Refer to RSU-15, "
 Exploded View
 "
 .
 8. Remove drive shaft (4WD). Refer to RAX-14, "
 Exploded View
 "
 .
 9. Remove coil spring from vehicle.
''',
'''
3. Remove steering wheel bolt after steering is locked.
4. Disconnect harness connector.
5. Remove steering wheel with the steering wheel puller
''',
'''
1. Set vehicle to the straight-ahead position.
''',
'''
1. Remove outer socket lock nut and remove outer socket.
2. Remove inner and outer boot clamps and remove boot.
''',
'''
 • Move joint up/down, left/right, and in the axial direction. Check for any rough movement or significant looseness.
 • Check boot for cracks or other damage, and also for grease leak
age.
 • If a malfunction is found, disassemble drive shaft, and then replace
 with new one.
''',
'''
Insert locating pin into vehicle side to position accelerator pedal assembly. Install mounting bolt to accelerator pedal assembly.
''',
'''
 Components to pay attention to include:
 1. Shifter assembly cover to finisher
 2. A/C control unit and cluster lid C
 3. Wiring harnesses behind audio and A/C control unit
'''
]

# Generate responses

In [ ]:
FILES_DIR = r'./data'

In [10]:
def load_db(files_dir):
    parser = Parser()
    emb_fn = CustomEmbeddingFunction()

    client = chromadb.Client()
    collection = client.create_collection(name="manuals", embedding_function=emb_fn)
    collection = fill_collection(collection, files_dir, parser)
    
    return collection

In [11]:
def load_model():
    model = Model()
    return model

In [12]:
db = load_db(FILES_DIR)
model = load_model()

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 29845.62it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.67s/it]
Device set to use cuda:0


In [13]:
responses = []

In [16]:
for query in queries:
    rag_input = '\n'.join(db.query(
                query_texts=[query], 
                n_results=1
            )['documents'][0])
    response = model.generate(query=query, rag_input=rag_input)
    responses.append(response)

c:\Users\adamm\Desktop\HSE\RAG\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [35]:
responses_wo_rag = []

In [36]:
for query in queries:
    response = model.generate(query=query)
    responses_wo_rag.append(response)

c:\Users\adamm\Desktop\HSE\RAG\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


# Example

In [42]:
queries[1]

'Rear seat belt warning lamp does not illuminate. How to diagnost it?'

In [43]:
print(responses_wo_rag[1])

If the rear seat belt warning lamp does not illuminate, it could be due to several reasons. Here are some steps to diagnose and potentially fix the issue:

1. **Check the Fuse:**
   - Locate the fuse box, which is usually found under the dashboard or in the engine compartment.
   - Refer to the vehicle's manual to find the fuse that controls the rear seat belt warning lamp.
   - Remove the fuse and check if it is blown. If it is, replace it with a fuse of the same rating.

2. **Inspect the Bulb:**
   - The bulb itself might be burnt out. Remove the bulb from the back of the lamp and check if it is working.
   - If the bulb is burnt out, replace it with a new one of the same type (e.g., incandescent, LED).

3. **Check the Wiring:**
   - Inspect the wiring leading to the rear seat belt warning lamp for any visible damage, such as cuts, frays, or corrosion.
   - If you find any damaged wiring, repair or replace the affected section.

4. **Test the Switch:**
   - The switch that activates 

In [44]:
print(responses[1])

1. Check DTC of "Air Bag Diagnosis Sensor Unit" in "Self Diagnosis Result" using CONSULT. Is DTC detected?
   - YES: Perform the trouble diagnosis related to the detected DTC. Refer to SRC-19, "DTC Index".
   - NO: Go to step 2.

2. Check DTC of "Combination Meter" in "Self Diagnosis Result" using CONSULT. Is DTC detected?
   - YES: Perform the trouble diagnosis related to the detected DTC. Refer to MWI-40, "DTC Index".
   - NO: Go to step 3.

3. Check Combination Meter power supply and ground circuit. Refer to MWI-66, "Combination Meter: Diagnosis Procedure". Is the inspection result normal?
   - YES: Go to step 4.
   - NO: Repair or replace the malfunctioning parts.

4. Check harness connector for disconnection, looseness or damage. Is the inspection result normal?
   - YES: Go to step 5.
   - NO: If damage, replace malfunctioning parts. If disconnection or looseness, securely lock the connector.

5. Check wiring harness externals. Is the inspection result normal?
   - YES: Go to ste

In [45]:
print(gts[1])


Diagnosis Procedure
 1.CHECK DTC OF “AIR BAG DIAGNOSIS SENSOR UNIT”     
Check DTC in “Self Diagnosis Result” “Air bag diagnosis sensor unit” using CONSULT.
 Is DTC detected?
 YES >>Perform the trouble diagnosis related to the detected DTC. Refer to SRC-19, "
 DTC
 Index
 "
 .
 NO >>GO TO 2.
 2.CHECK DTC OF “COMBINATION METER” 
Check DTC in “Self Diagnosis Result” “Air bag diagnosis sensor unit” using CONSULT.
 Is DTC detected?
 YES >>Perform the trouble diagnosis related to the detected DTC. Refer to MWI-40, "
 DTC Index
 "
 .
 NO >>GO TO 3
 3.CHECK COMBINATION METER POWER SUPPLY AND GROUND CIRCUIT
 Check combination meter power supply and ground circuit. Refer to MWI-66, "
 COMBINATION METER : Diag
nosis Procedure
 "
 .
 Is the inspection result normal?
 YES >>GO TO 4.
 NO >>Repair or replace the malfunctioning parts.
 4.CHECK HARNESS CONNECTOR
 Check the harness connector for disconnection, looseness or damage.
 Is the inspection result normal?
 YES >>GO TO 5.
 NO-1 >>damage: Repla

# Metrics

In [26]:
from evaluate import load
import numpy as np

In [5]:
bertscore = load("bertscore")

In [37]:
metrics_wo_rag = bertscore.compute(predictions=responses_wo_rag, references=gts, lang="en")

In [38]:
print(f"Precision: {np.array(metrics_wo_rag['precision']).mean():.3f}")
print(f"Recall: {np.array(metrics_wo_rag['recall']).mean():.3f}")
print(f"F1: {np.array(metrics_wo_rag['f1']).mean():.3f}")

Precision: 0.788
Recall: 0.840
F1: 0.812


In [21]:
metrics = bertscore.compute(predictions=responses, references=gts, lang="en")

In [29]:
print(f"Precision: {np.array(metrics['precision']).mean():.3f}")
print(f"Recall: {np.array(metrics['recall']).mean():.3f}")
print(f"F1: {np.array(metrics['f1']).mean():.3f}")

Precision: 0.870
Recall: 0.903
F1: 0.886
